# ハンズオンラボ：Databricksでエージェントシステムを構築する

## パート2 - エージェント評価
エージェントを作成したので、そのパフォーマンスをどのように評価するのでしょうか？
第2部では、評価に焦点を当てるために製品サポートエージェントを作成します。
このエージェントは、RAGアプローチを使用して製品ドキュメントを活用し、製品に関する質問に回答します。

### 2.1 新しいエージェントとリトリーバーツールの定義
- [**agent.py**]($./agent.py)：サンプルエージェントが設定されています。まずこのファイルを確認し、構成要素を理解しましょう
- **ベクター検索**：特定の製品に関連するドキュメントを検索できるベクター検索エンドポイントを作成しました。
- **リトリーバー関数の作成**：リトリーバーのプロパティを定義し、LLMから呼び出せるようにパッケージ化します。

### 2.2 評価データセットの作成
- サンプルの評価データセットを用意していますが、[合成的に生成](https://www.databricks.com/jp/blog/streamline-ai-agent-evaluation-with-new-synthetic-data-capabilities)することも可能です。

### 2.3 MLflow.evaluate() の実行
- MLflowは評価データセットを使ってエージェントの応答をテストします
- LLMジャッジが出力をスコア化し、すべてを見やすいUIでまとめます

### 2.4 必要な改善を行い、再評価を実施
- 評価結果からフィードバックを得てリトリーバー設定を変更
- 再度評価を実施し、改善を確認しましょう！

In [0]:
%pip install -U -qqqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()

In [0]:
catalog_name = "handson"
system_schema_name = "bricks_hr"

## 環境変数を通じたエージェントの設定

環境変数から`agent.py`のパラメータを設定します。

In [0]:
from databricks.sdk import WorkspaceClient
import os
import re

# ワークスペースクライアントを使用して現在のユーザーに関する情報を取得
w = WorkspaceClient()
user_email = w.current_user.me().emails[0].value
username = user_email.split('@')[0]
username = re.sub(r'[^a-zA-Z0-9_]', '_', username) # 特殊文字をアンダースコアに置換

# スキーマを指定します
user_schema_name = f"agents_lab_{username}" # ユーザーごとのスキーマ

In [0]:
import os

# LLMエンドポイント名
os.environ["LLM_ENDPOINT_NAME"] = "databricks-gpt-oss-20b"

# Vector Search名
os.environ["VS_NAME"] = f"{catalog_name}.{system_schema_name}.hr_manuals_index"

print("環境変数を設定しました:")
print(f"LLM_ENDPOINT_NAME: {os.environ.get('LLM_ENDPOINT_NAME')}")
print(f"VS_NAME: {os.environ.get('VS_NAME')}")

In [0]:
from agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "勤務間のインターバルは何時間以上が推奨されてますか？"}]})

In [0]:
AGENT.predict({"messages": [{"role": "user", "content": "こんにちは！"}]})

In [0]:
AGENT.predict({"messages": [{"role": "user", "content": "彼女に振られてしまいました。どうすれば仲直りできるかな？"}]})

In [0]:
from IPython.display import Image, display

# エージェントのグラフ構造を可視化
display(Image(AGENT.agent.get_graph().draw_mermaid_png()))

## `agent` をMLflowモデルとしてログに記録する
[agent]($./agent)ノートブックのコードとしてエージェントをログに記録します。詳細は[MLflow - コードからのモデル](https://mlflow.org/docs/latest/models.html#models-from-code)を参照してください。

In [0]:
# デプロイ時に自動認証パススルーを指定するためのDatabricksリソースを決定
import mlflow
from agent import tools, LLM_ENDPOINT_NAME, VS_NAME
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint, DatabricksVectorSearchIndex
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
             DatabricksVectorSearchIndex(index_name=VS_NAME),
             DatabricksServingEndpoint(endpoint_name="databricks-gte-large-en")]

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "勤務間のインターバルは何時間以上が推奨されてますか？"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            "databricks-connect"
        ]
    )

In [0]:
# モデルをロードし、予測関数を作成
logged_model_uri = f"runs:/{logged_agent_info.run_id}/agent"
loaded_model = mlflow.pyfunc.load_model(logged_model_uri)

def predict_wrapper(query):
    # チャット形式モデル用の入力を整形
    model_input = {
        "messages": [{"role": "user", "content": query}]
    }
    response = loaded_model.predict(model_input)
    
    messages = response['messages']
    return messages[-1]['content']

## エージェントを[エージェント評価](https://docs.databricks.com/aws/ja/generative-ai/agent-evaluation)で評価する

評価データセットのリクエストや期待される応答を編集し、エージェントを反復しながら評価を実行し、mlflowを活用して計算された品質指標を追跡できます。

In [0]:
import pandas as pd

data = {
    "request": [
        "このマニュアルの目的は何ですか？",
        "採用業務マニュアルの主な利用者は誰ですか？",
        "退職時に返却すべき物品にはどのようなものがありますか？",
        "アクセス権の停止はどのタイミングで行われますか？",
        "給与の支給タイミングはどのように定められていますか？",
        "人事労務管理がIT企業において特に重要とされる理由は何ですか？",
        "情報機器やデータの返却時に重視すべきセキュリティ対策は何ですか？",
        "採用プロセスの中で書類選考が重要とされる理由は何ですか？",
        "在宅勤務時の情報セキュリティ対策はどのように実施すべきですか？",
        "社外から人材紹介を受ける場合の契約書締結手順はどのようになりますか？"
    ],
    "expected_facts": [
        [
            "本マニュアルは、従業員と組織の持続可能な成長を実現するための実践的な業務ガイドブックである。",
            "人事労務管理を単なる事務処理ではなく、従業員が安心して働き成長するための基盤と位置付けている。",
            "組織の競争力向上と持続的発展を支える戦略的な役割を担っている。"
        ],
        [
            "主な利用者は、人事部新任担当者、現場管理職、経営陣である。",
            "人事部新任者は採用業務全体を理解するため、現場管理職は面接官として評価基準を確認するため、経営陣は採用戦略の妥当性を確認するために利用する。"
        ],
        [
            "退職時にはノートPC、スマートフォン、USBメモリ、ICカード、入館証、社内アカウント、契約書類、名刺などを返却する必要がある。",
            "情報機器だけでなく、物理的セキュリティ関連物品や業務関連資料も含まれる。"
        ],
        [
            "退職申出時に機密情報へのアクセス制限を行う。",
            "引継ぎ期間中は必要最小限のアクセス権のみを維持する。",
            "最終出勤日には全アクセス権を即座に停止し、退職日にアカウントを完全削除する。"
        ],
        [
            "通常給与は通常の支給日に支払われる。",
            "退職金は退職後1ヶ月以内に支給される。",
            "精算金は確定次第速やかに支給または返金される。"
        ],
        [
            "IT業界では技術革新や働き方の変化が激しいため、従来の人事管理では対応できない課題が多い。",
            "人事労務管理は社員の安心と成長を支える基盤であり、企業競争力の向上に直結する。"
        ],
        [
            "退職時には個人フォルダのバックアップと機密データの完全消去を実施する。",
            "メールや認証情報の引継ぎ・無効化を行う。",
            "データアクセスログやファイル操作履歴などの監査証跡を保持する。"
        ],
        [
            "書類選考は短時間で候補者を適切に評価し、公正な採用を実現するために重要である。",
            "選考の品質・効率・法的適正性を確保する目的がある。"
        ],
        [
            "在宅勤務に関する明確な記載はないが、一般的にVPN接続やアクセス制御の強化、情報持ち出しの禁止などが必要である。",
            "機密データを扱う際は、クラウドサービスのアクセスログ監査を実施することが望ましい。"
        ],
        [
            "本マニュアルには具体的な人材紹介契約書締結の手順は記載されていない。",
            "一般的には、紹介契約時に報酬条件や独占条項、秘密保持契約を明記した文書を締結する必要がある。",
            "社外取引に関しては人事部と法務部の承認が必須である。"
        ]
    ]
}

eval_dataset = pd.DataFrame(data)
display(eval_dataset)

LLMジャッジである[スコアラー](https://docs.databricks.com/gcp/ja/mlflow3/genai/eval-monitor/custom-judge/meets-guidelines)を定義します。

In [0]:
from mlflow.genai.scorers import (
    Correctness,
    Guidelines,
    RelevanceToQuery,
    RetrievalGroundedness,
    RetrievalRelevance,
    RetrievalSufficiency,
    Safety,
)
import mlflow.genai

# 評価用データセットを作成
eval_data = []
for request, facts in zip(data["request"], data["expected_facts"]):
    eval_data.append({
        "inputs": {
            "query": request  # 関数の引数と一致させる
        },
        "expected_response": "\n".join(facts)
    })

# 評価用スコアラーを定義
# LLMジャッジが応答を評価するためのガイドライン

# 製品情報評価に特化したカスタムスコアラーを定義
scorers = [
    Correctness(),
    # RelevanceToQuery(),
    # RetrievalGroundedness(),
    # RetrievalRelevance(),
    RetrievalSufficiency(),
    # Safety(),
]

評価を実行します。

In [0]:
print("評価を実行中...")
with mlflow.start_run():
    results = mlflow.genai.evaluate(
        data=eval_data,
        predict_fn=predict_wrapper, 
        scorers=scorers,
    )

## [agent.py]($./agent.py) ファイルに戻り、評価結果を向上させるための変更をを減らしましょう。

In [0]:
with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            "databricks-connect"
        ]
    )

# モデルをロードし、予測関数を作成
logged_model_uri = f"runs:/{logged_agent_info.run_id}/agent"
loaded_model = mlflow.pyfunc.load_model(logged_model_uri)

def predict_wrapper(query):
    # チャット形式モデル用の入力を整形
    model_input = {
        "messages": [{"role": "user", "content": query}]
    }
    response = loaded_model.predict(model_input)
    
    messages = response['messages']
    return messages[-1]['content']
  
print("評価を実行中...")
with mlflow.start_run():
    results = mlflow.genai.evaluate(
        data=eval_data,
        predict_fn=predict_wrapper, 
        scorers=scorers,
    )

## モデルをUnity Catalogに登録する

以下の `catalog`、`schema`、`model_name` を更新して、MLflowモデルをUnity Catalogに登録します。

In [0]:
mlflow.set_registry_uri("databricks-uc")

# UCモデル用のカタログ、スキーマ、モデル名を定義
model_name = "operation_agent"
UC_MODEL_NAME = f"{catalog_name}.{system_schema_name}.{model_name}"

# モデルをUCに登録
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

モデルバージョンにアクセスし、**依存関係**タブでエージェントのリネージを確認してみましょう。

In [0]:
from IPython.display import display, HTML

# DatabricksのホストURLを取得
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# 作成したエージェントへのHTMLリンクを作成
html_link = f'<a href="https://{workspace_url}/explore/data/models/{catalog_name}/{user_schema_name}/product_agent" target="_blank">登録済みエージェントをUnity Catalogで表示</a>'
display(HTML(html_link))

## エージェントのデプロイ

上で使用した環境変数を設定してエージェントをモデルサービングエンドポイントにデプロイします。

In [0]:
from databricks import agents

# 環境変数を辞書として定義
environment_vars = {
    "LLM_ENDPOINT_NAME": os.environ["LLM_ENDPOINT_NAME"],
    "VS_NAME": os.environ["VS_NAME"],
}

# モデルをレビューアプリおよびモデルサービングエンドポイントにデプロイ
agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags={"endpointSource": "Agent Lab"},
    environment_vars=environment_vars,
    timeout=900,  # 15分に延長
)

## 今後の方向性

- 更なる[評価](https://docs.databricks.com/aws/ja/mlflow3/genai/getting-started/eval)と[監視](https://docs.databricks.com/aws/ja/mlflow3/genai/eval-monitor/)を通じた改善
- [アプリ](https://docs.databricks.com/aws/ja/dev-tools/databricks-apps/get-started)との連携